In [ ]:
import pandas as pd
wine = pd.read_csv('https://bit.ly/wine_csv_data')

In [ ]:
data = wine[['alcohol','sugar','pH']].to_numpy()
target = wine['class'].to_numpy()

In [ ]:
from sklearn.model_selection import train_test_split
train_input,test_input,train_target,test_target = train_test_split(data,target,test_size=0.2,random_state=42)

훈련세트 , 테스트 세트 나누는 것 

In [ ]:
sub_input , val_input , sub_target, val_target = train_test_split(train_input,train_target,test_size=0.2 , random_state=42)

훈련세트와 검증세트를 나누는 것

In [ ]:
print(sub_input.shape,val_input.shape)

In [ ]:
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(random_state = 42)
dt.fit(sub_input,sub_target)
print(dt.score(sub_input,sub_target))
print(dt.score(val_input,val_target))

위에서 봤듯이 검증세트를 만들면 훈련세트가 줄어든다 따라서 이를 보완하기위해서 교차검증을 실시한다 이를 k-fold cross validation

In [ ]:
from sklearn.model_selection import cross_validate 
scores = cross_validate(dt,train_input,train_target)
print(scores)

In [ ]:
import numpy as np
print(np.mean(scores['test_score']))

교차검증의 최종 점수는 test_score 키에 담긴 5개의 점수를 평균하여 구할 수 있다. 

In [ ]:
from sklearn.model_selection import StratifiedKFold
scores = cross_validate(dt,train_input, train_target,cv=StratifiedKFold())
print(np.mean(scores['test_score']))

cross_validate() 함수는 기본적으로 회귀 모델일 경우 KFold 분할기를 사용하고 / 분류모델일 경우 타깃 클래스를 골고루 나누기 위해 StratifiedKFold를 사용한다.


In [ ]:
splitter = StratifiedKFold(n_splits = 10 , shuffle=True , random_state=42)
scores = cross_validate(dt,train_input,train_target , cv=splitter)
print(np.mean(scores['test_score']))

훈련세트를 섞은 후 10-폴드 교차검증을 수행하는 것 

In [ ]:
from sklearn.model_selection import GridSearchCV
params = {'min_impurity_decrease' : [0.0001,0.0002,0.0003,0.0004,0.0005]}
gs = GridSearchCV(DecisionTreeClassifier(random_state=42),params,n_jobs=-1)
gs.fit(train_input,train_target)

In [ ]:
dt=gs.best_estimator_
print(dt.score(train_input,train_target))
print(gs.best_params_)
print(gs.cv_results_['mean_test_score'])

In [ ]:
from scipy.stats import uniform,randint
rgen = randint(0,10)
rgen.rvs(10)

randint같은 경우는 샘플링하는 것이다.

In [ ]:
np.unique(rgen.rvs(1000),return_counts=True)

In [ ]:
ugen=uniform(0,1)
ugen.rvs(10)

난수 발생기라고 생각하면 된다.

In [ ]:
params = {'min_impurity_decrease' : uniform(0.0001,0.001),
          'max_depth' : randint(20,50),
          'min_samples_split' : randint(2,25),
          'min_samples_leaf' : randint(1,25),}
          

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
gs = RandomizedSearchCV(DecisionTreeClassifier(random_state = 42),params,n_iter=100,n_jobs=-1,random_state=42)
gs.fit(train_input,train_target)


In [ ]:
print(gs.best_params_)
print(np.max(gs.cv_results_['mean_test_score']))

In [ ]:
dt = gs.best_estimator_
print(dt.score(test_input,test_target))